In [1]:
HOMEPATH = "/home/ubuntu/fastai/"
DATAPATH = HOMEPATH + "data/Kaggle_digit_recognizer/"
print("HOMEPATH:", HOMEPATH)
print("DATAPATH:", DATAPATH)
import os

('HOMEPATH:', '/home/ubuntu/fastai/')
('DATAPATH:', '/home/ubuntu/fastai/data/Kaggle_digit_recognizer/')


In [2]:
os.chdir(HOMEPATH)
print ("os.getcwd:", os.getcwd())
import utils; reload(utils)
from utils import *

%matplotlib inline

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from __future__ import division, print_function

from keras.utils.np_utils import to_categorical

('os.getcwd:', '/home/ubuntu/fastai')


WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5110)
Using Theano backend.


In [3]:
print ("keras.__version__:", keras.__version__)
print ("theano.__version__:", theano.__version__)

keras.__version__: 1.2.2
theano.__version__: 0.9.0.dev-c697eeab84e5b8a74908da654b66ec9eca4f1291


In [4]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
#X_train, X_test, y_train, y_test = train_test_split(pixelData, labelData, test_size=0.1, random_state=42)#split was 0.33
print ("X_train:", type(X_train), X_train.shape)
print ("y_train:", type(y_train), y_train.shape)
print ("X_test:", type(X_test), X_test.shape)
print ("y_test:", type(y_test), y_test.shape)

X_train: <type 'numpy.ndarray'> (60000, 28, 28)
y_train: <type 'numpy.ndarray'> (60000,)
X_test: <type 'numpy.ndarray'> (10000, 28, 28)
y_test: <type 'numpy.ndarray'> (10000,)


In [5]:
if len(X_test.shape)==3:
    print("expand shape")
    X_test = np.expand_dims(X_test,1)
    X_train = np.expand_dims(X_train,1)
else:
        print("expand shape already done, don't do again.")

print ("type(X_test):", type(X_test), X_test.shape)
print ("type(X_train):", type(X_train), X_train.shape)

expand shape
type(X_test): <type 'numpy.ndarray'> (10000, 1, 28, 28)
type(X_train): <type 'numpy.ndarray'> (60000, 1, 28, 28)


In [6]:
print ("X_train:", X_train.shape)
print ("X_test:", X_test.shape)


X_train: (60000, 1, 28, 28)
X_test: (10000, 1, 28, 28)


In [7]:
print ("type(y_train):", type(y_train), y_train.shape, len(y_train.shape))
print ("type(y_test):", type(y_test), y_test.shape, len(y_test.shape))

type(y_train): <type 'numpy.ndarray'> (60000,) 1
type(y_test): <type 'numpy.ndarray'> (10000,) 1


In [8]:
if len(y_test.shape)==1:
    y_train = to_categorical(y_train)
else:
    print ("y_train already onehot'd, dont try again.")

if len(y_test.shape)==1:
    y_test = to_categorical(y_test)
else:
    print ("y_train already onehot'd, dont try again.")

print ("type(y_train):", type(y_train), y_train.shape)
print ("type(y_test):", type(y_test), y_test.shape)

type(y_train): <type 'numpy.ndarray'> (60000, 10)
type(y_test): <type 'numpy.ndarray'> (10000, 10)


In [9]:
print ("y_train\n", y_train[0:5, :])
print ("y_test\n", y_test[0:5, :])

y_train
 [[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]]
y_test
 [[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]]


In [10]:
mean_px = X_train.mean().astype(np.float32)
std_px = X_train.std().astype(np.float32)

In [11]:
def norm_input(x): return (x-mean_px)/std_px

In [12]:
os.chdir(HOMEPATH)
print ("os.getcwd:", os.getcwd())
import vgg16bn

os.getcwd: /home/ubuntu/fastai


In [26]:
def showLayersInfo(model):
    print ("Number of layers : ", len(model.layers))
    count = 0
    for layer in model.layers:
        print (count, type(layer), ", trainable:", layer.trainable)
        print ("input:", layer.input_shape, ", output:",layer.output_shape, ", len(weights)", len(layer.get_weights()))
        print ("layer.output:", type(layer.output))
        print ()
        count +=1

In [22]:
def get_model_bn_do():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),#1

        #ConvBlock(2, 64) = 2 sets of layers with Convolution2D.filters=64
        ZeroPadding2D((1, 1)),#2
        Convolution2D(64, 3, 3, activation='relu'),#3
        ZeroPadding2D((1, 1)),#4
        Convolution2D(64, 3, 3, activation='relu'),
        MaxPooling2D((2, 2), strides=(2, 2)),

        #ConvBlock(2, 128) = 2 sets of layers with Convolution2D.filters=64
        ZeroPadding2D((1, 1)),
        Convolution2D(128, 3, 3, activation='relu'),
        ZeroPadding2D((1, 1)),
        Convolution2D(128, 3, 3, activation='relu'),
        MaxPooling2D((2, 2), strides=(2, 2)),

        #ConvBlock(3, 256) = 3 sets of layers with Convolution2D.filters=256
        ZeroPadding2D((1, 1)),
        Convolution2D(256, 3, 3, activation='relu'),
        ZeroPadding2D((1, 1)),
        Convolution2D(256, 3, 3, activation='relu'),
        ZeroPadding2D((1, 1)),
        Convolution2D(256, 3, 3, activation='relu'),
        MaxPooling2D((2, 2), strides=(2, 2)),

        #ConvBlock(3, 512) = 3 sets of layers with Convolution2D.filters=512
        ZeroPadding2D((1, 1)),
        Convolution2D(512, 3, 3, activation='relu'),
        ZeroPadding2D((1, 1)),
        Convolution2D(512, 3, 3, activation='relu'),
        ZeroPadding2D((1, 1)),
        Convolution2D(512, 3, 3, activation='relu'),
        MaxPooling2D((2, 2), strides=(2, 2)),


        #NB: for some weird reason, if a second block of ConvBlock(3, 512) is attempted, errors result.
        Flatten(),
        
        #FCBlock
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),

        #FCBlock
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),


        Dense(10, activation='softmax')#last dense layer - 10 categories for 10 digits.

        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [23]:
model1 = get_model_bn_do()
model1.summary()


/usr/local/lib/python2.7/dist-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_3 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_3 (Lambda)                (None, 1, 28, 28)     0           lambda_input_3[0][0]             
____________________________________________________________________________________________________
zeropadding2d_27 (ZeroPadding2D) (None, 1, 30, 30)     0           lambda_3[0][0]                   
____________________________________________________________________________________________________
convolution2d_27 (Convolution2D) (None, 64, 28, 28)    640         zeropadding2d_27[0][0]           
____________________________________________________________________________________________________
zeropadding2d_28 (ZeroPadding2D) (None, 64, 30, 30)    0           convolution2d_27[0][0]           
___________________________________________________________________________________________

In [24]:
showLayersInfo(model1)

Number of layers :  33
0 <class 'keras.layers.core.Lambda'> , trainable: True
input: (None, 1, 28, 28) , output: (None, 1, 28, 28) , len(weights) 0 

layer.output: <class 'theano.tensor.var.TensorVariable'>
1 <class 'keras.layers.convolutional.ZeroPadding2D'> , trainable: True
input: (None, 1, 28, 28) , output: (None, 1, 30, 30) , len(weights) 0 

layer.output: <class 'theano.tensor.var.TensorVariable'>
2 <class 'keras.layers.convolutional.Convolution2D'> , trainable: True
input: (None, 1, 30, 30) , output: (None, 64, 28, 28) , len(weights) 2 

layer.output: <class 'theano.tensor.var.TensorVariable'>
3 <class 'keras.layers.convolutional.ZeroPadding2D'> , trainable: True
input: (None, 64, 28, 28) , output: (None, 64, 30, 30) , len(weights) 0 

layer.output: <class 'theano.tensor.var.TensorVariable'>
4 <class 'keras.layers.convolutional.Convolution2D'> , trainable: True
input: (None, 64, 30, 30) , output: (None, 64, 28, 28) , len(weights) 2 

layer.output: <class 'theano.tensor.var.Tenso

In [17]:
def get_model_2():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),

        #ConvBlock(2, 64) = 2 sets of layers with Convolution2D.filters=64
        ZeroPadding2D((1, 1)),
        Convolution2D(64, 3, 3, activation='relu'),
        ZeroPadding2D((1, 1)),
        Convolution2D(64, 3, 3, activation='relu'),
        MaxPooling2D((2, 2), strides=(2, 2)),

        #ConvBlock(2, 128) = 2 sets of layers with Convolution2D.filters=64
        ZeroPadding2D((1, 1)),
        Convolution2D(128, 3, 3, activation='relu'),
        ZeroPadding2D((1, 1)),
        Convolution2D(128, 3, 3, activation='relu'),
        MaxPooling2D((2, 2), strides=(2, 2)),

        #ConvBlock(3, 256) = 3 sets of layers with Convolution2D.filters=256
        ZeroPadding2D((1, 1)),
        Convolution2D(256, 3, 3, activation='relu'),
        ZeroPadding2D((1, 1)),
        Convolution2D(256, 3, 3, activation='relu'),
        ZeroPadding2D((1, 1)),
        Convolution2D(256, 3, 3, activation='relu'),
        MaxPooling2D((2, 2), strides=(2, 2)),

        #ConvBlock(3, 512) = 3 sets of layers with Convolution2D.filters=512
        ZeroPadding2D((1, 1)),
        Convolution2D(512, 3, 3, activation='relu'),
        ZeroPadding2D((1, 1)),
        Convolution2D(512, 3, 3, activation='relu'),
        ZeroPadding2D((1, 1)),
        Convolution2D(512, 3, 3, activation='relu'),
        MaxPooling2D((2, 2), strides=(2, 2)),

        #ConvBlock(3, 512) = 3 sets of layers with Convolution2D.filters=512
        ZeroPadding2D((1, 1)),
        Convolution2D(512, 3, 3, activation='relu'),
        ZeroPadding2D((1, 1)),
        Convolution2D(512, 3, 3, activation='relu'),
        ZeroPadding2D((1, 1)),
        Convolution2D(512, 3, 3, activation='relu'),
        #MaxPooling2D((2, 2), strides=(2, 2)),

        #ConvBlock(3, 512) = 3 sets of layers with Convolution2D.filters=512
        ZeroPadding2D((1, 1)),
        Convolution2D(512, 3, 3, activation='relu'),
        ZeroPadding2D((1, 1)),
        Convolution2D(512, 3, 3, activation='relu'),
        ZeroPadding2D((1, 1)),
        Convolution2D(512, 3, 3, activation='relu'),


        #NB: for some weird reason, if a second block of ConvBlock(3, 512) is attempted, errors result.
        Flatten(),
        
        #FCBlock
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),

        #FCBlock
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),


        Dense(10, activation='softmax')#last dense layer - 10 categories for 10 digits.

        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [18]:
model2 = get_model_2()
model2.summary()

/usr/local/lib/python2.7/dist-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_2 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_2 (Lambda)                (None, 1, 28, 28)     0           lambda_input_2[0][0]             
____________________________________________________________________________________________________
zeropadding2d_11 (ZeroPadding2D) (None, 1, 30, 30)     0           lambda_2[0][0]                   
____________________________________________________________________________________________________
convolution2d_11 (Convolution2D) (None, 64, 28, 28)    640         zeropadding2d_11[0][0]           
____________________________________________________________________________________________________
zeropadding2d_12 (ZeroPadding2D) (None, 64, 30, 30)    0           convolution2d_11[0][0]           
___________________________________________________________________________________________

dense_5 (Dense)                  (None, 4096)          16781312    dropout_3[0][0]                  
____________________________________________________________________________________________________
batchnormalization_4 (BatchNorma (None, 4096)          16384       dense_5[0][0]                    
____________________________________________________________________________________________________
dropout_4 (Dropout)              (None, 4096)          0           batchnormalization_4[0][0]       
____________________________________________________________________________________________________
dense_6 (Dense)                  (None, 10)            40970       dropout_4[0][0]                  
Total params: 40,749,258
Trainable params: 40,732,874
Non-trainable params: 16,384
____________________________________________________________________________________________________


In [27]:
showLayersInfo(model2)

Number of layers :  45
0 <class 'keras.layers.core.Lambda'> , trainable: True
input: (None, 1, 28, 28) , output: (None, 1, 28, 28) , len(weights) 0
layer.output: <class 'theano.tensor.var.TensorVariable'>

1 <class 'keras.layers.convolutional.ZeroPadding2D'> , trainable: True
input: (None, 1, 28, 28) , output: (None, 1, 30, 30) , len(weights) 0
layer.output: <class 'theano.tensor.var.TensorVariable'>

2 <class 'keras.layers.convolutional.Convolution2D'> , trainable: True
input: (None, 1, 30, 30) , output: (None, 64, 28, 28) , len(weights) 2
layer.output: <class 'theano.tensor.var.TensorVariable'>

3 <class 'keras.layers.convolutional.ZeroPadding2D'> , trainable: True
input: (None, 64, 28, 28) , output: (None, 64, 30, 30) , len(weights) 0
layer.output: <class 'theano.tensor.var.TensorVariable'>

4 <class 'keras.layers.convolutional.Convolution2D'> , trainable: True
input: (None, 64, 30, 30) , output: (None, 64, 28, 28) , len(weights) 2
layer.output: <class 'theano.tensor.var.TensorVaria

In [ ]:
def get_model_3():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),

        #ConvBlock(2, 64) = 2 sets of layers with Convolution2D.filters=64
        ZeroPadding2D((1, 1)),
        Convolution2D(64, 3, 3, activation='relu'),
        ZeroPadding2D((1, 1)),
        Convolution2D(64, 3, 3, activation='relu'),
        MaxPooling2D((2, 2), strides=(2, 2)),

        #ConvBlock(2, 128) = 2 sets of layers with Convolution2D.filters=64
        ZeroPadding2D((1, 1)),
        Convolution2D(128, 3, 3, activation='relu'),
        ZeroPadding2D((1, 1)),
        Convolution2D(128, 3, 3, activation='relu'),
        MaxPooling2D((2, 2), strides=(2, 2)),

        #ConvBlock(3, 256) = 3 sets of layers with Convolution2D.filters=256
        ZeroPadding2D((1, 1)),
        Convolution2D(256, 3, 3, activation='relu'),
        ZeroPadding2D((1, 1)),
        Convolution2D(256, 3, 3, activation='relu'),
        ZeroPadding2D((1, 1)),
        Convolution2D(256, 3, 3, activation='relu'),
        MaxPooling2D((2, 2), strides=(2, 2)),

        #ConvBlock(3, 512) = 3 sets of layers with Convolution2D.filters=512
        ZeroPadding2D((1, 1)),
        Convolution2D(512, 3, 3, activation='relu'),
        ZeroPadding2D((1, 1)),
        Convolution2D(512, 3, 3, activation='relu'),
        ZeroPadding2D((1, 1)),
        Convolution2D(512, 3, 3, activation='relu'),
        MaxPooling2D((2, 2), strides=(2, 2)),

        #ConvBlock(3, 512) = 3 sets of layers with Convolution2D.filters=512
        ZeroPadding2D((1, 1)),
        Convolution2D(512, 3, 3, activation='relu'),
        ZeroPadding2D((1, 1)),
        Convolution2D(512, 3, 3, activation='relu'),
        ZeroPadding2D((1, 1)),
        Convolution2D(512, 3, 3, activation='relu'),
        MaxPooling2D((2, 2), strides=(2, 2))


        #NB: for some weird reason, if a second block of ConvBlock(3, 512) is attempted, errors result.
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model3 = get_model_3()
model3.summary()

In [ ]:
showLayersInfo(model3)

Model 1  
23 <class 'keras.layers.convolutional.Convolution2D'> , trainable: True  
input: (None, 512, 5, 5) , output: (None, 512, 3, 3) , len(weights) 2   

24 <class 'keras.layers.pooling.MaxPooling2D'> , trainable: True  
input: (None, 512, 3, 3) , output: (None, 512, 1, 1) , len(weights) 0   

25 <class 'keras.layers.core.Flatten'> , trainable: True  
input: (None, 512, 1, 1) , output: (None, 512) , len(weights) 0   



model 2  

fails  

model 3  

23 <class 'keras.layers.convolutional.Convolution2D'> , trainable: True  
input: (None, 512, 5, 5) , output: (None, 512, 3, 3) , len(weights) 2   

24 <class 'keras.layers.pooling.MaxPooling2D'> , trainable: True  
input: (None, 512, 3, 3) , output: (None, 512, 1, 1) , len(weights) 0   

25 <class 'keras.layers.convolutional.ZeroPadding2D'> , trainable: True  
input: (None, 512, 1, 1) , output: (None, 512, 3, 3) , len(weights) 0   

26 <class 'keras.layers.convolutional.Convolution2D'> , trainable: True  
input: (None, 512, 3, 3) , output: (None, 512, 1, 1) , len(weights) 2   

27 <class 'keras.layers.convolutional.ZeroPadding2D'> , trainable: True  
input: (None, 512, 1, 1) , output: (None, 512, 3, 3) , len(weights) 0   

28 <class 'keras.layers.convolutional.Convolution2D'> , trainable: True  
input: (None, 512, 3, 3) , output: (None, 512, 1, 1) , len(weights) 2   

29 <class 'keras.layers.convolutional.ZeroPadding2D'> , trainable: True   
input: (None, 512, 1, 1) , output: (None, 512, 3, 3) , len(weights) 0   

30 <class 'keras.layers.convolutional.Convolution2D'> , trainable: True  
input: (None, 512, 3, 3) , output: (None, 512, 1, 1) , len(weights) 2   

31 <class 'keras.layers.pooling.MaxPooling2D'> , trainable: True  
input: (None, 512, 1, 1) , output: (None, 512, 0, 0) , len(weights) 0   


model 1 cf model 3
model 1: layer 23 = last Convolution2D, output: (None, 512, 3, 3)  
         layer 24 =      MaxPooling2D,  output: (None, 512, 1, 1)  
         layer 25 =      Flatten,       output: (None, 512)  


model 3: layer 30 = last Convolution2D, output: (None, 512, 1, 1)  
         layer 31 =      MaxPooling2D,  output: (None, 512, 0, 0)  
         error occurs at Flatten, hence Flatten layer not added.  
err msg :   
The shape of the input to "Flatten" is not fully defined (got (512, 0, 0).   
Make sure to pass a complete "input_shape" or "batch_input_shape"   
argument to the first layer in your model.  

